# ToDO

### 
1. Anchor paper-<br>
    a. State the anchor paper<br>
    b. What is the main objective of the paper, what are they trying to solve?<br>
    c. Provide a short summary of the approach presented in the paper<br>
    
2. Suggesting an improvement-<br>
    a. Suggest an improvement to the paper you chose. Give a detailed explanation of your
    proposed improvement and why you think it might improve the algorithm’s
    performance.<br>
    
3. Add Netflix Dataset<br>

4. Select a well-known algorithm for comparison(MF + Another one(topk?))<br> 
    a. Select a well-known baseline algorithm to compare with the algorithms you are
    evaluating. 
5.Hyper parameter optimization-<br>
    1. Do hyper parameter optimization for the algorithms that you selected for stages
        1,2 and 3.

# AutoRec cs3639 Recommendation Systems course IDC

### here will be general explanations

In [5]:
import numpy as np
import pandas as pd
import sklearn
import torch
from torch import nn

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


## datasets

In this project, we will use 2 datasets:
* **movielens**, which can be downloaded using `utils.datasets_download.py` or straight from [here](http://files.grouplens.org/datasets/movielens/).
* **netflixprize**, which can be downloaded from this [semi-parsed version from kaggle](https://www.kaggle.com/netflix-inc/netflix-prize-data) or from this [raw version](https://archive.org/download/nf_prize_dataset.tar)

**NOTE**: for the notebook to run properly, you should save you dataset under `data` folder and `movielens` folder for the movielens dataset and `netflix` folder for the netflixprize dataset.
i.e `data/movielens` folder and `data/netflix` folder respectively.

# Models implementation 

## MF

### data set preperation

In [8]:
from src.data_prep import movielens_load
train, test = movielens_load()
print(train.shape)
train

(90570, 4)


,user_id,item_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
90565,943,1047,2,875502146
90566,943,1074,4,888640250
90567,943,1188,3,888640250
90568,943,1228,3,888640275


### model

In [11]:
from src.matrixfactorization import MatrixFactorization
num_users = train.user_id.max()
num_items = train.item_id.max()
print(num_users, num_items)
model = MatrixFactorization(num_users, num_items).to(device)


943 1682


### train

In [12]:
from src.matrixfactorization import mf_train

# %%time
mf_train(train, model,device, epochs=1000, lr=0.001, reg=0.001)

train RMSE: 5.380555629730225
train RMSE: 5.375885486602783
train RMSE: 5.371224880218506
train RMSE: 5.366573810577393
train RMSE: 5.361932277679443
train RMSE: 5.357300281524658
train RMSE: 5.352678298950195
train RMSE: 5.348066806793213
train RMSE: 5.343465328216553
train RMSE: 5.338874340057373
train RMSE: 5.334293365478516
train RMSE: 5.329723834991455
train RMSE: 5.325164318084717
train RMSE: 5.320615768432617
train RMSE: 5.316078186035156
train RMSE: 5.311551570892334
train RMSE: 5.30703592300415
train RMSE: 5.302530765533447
train RMSE: 5.298036098480225
train RMSE: 5.293553352355957
train RMSE: 5.289081573486328
train RMSE: 5.28462028503418
train RMSE: 5.28016996383667
train RMSE: 5.275731563568115
train RMSE: 5.271303653717041
train RMSE: 5.2668867111206055
train RMSE: 5.262481689453125
train RMSE: 5.258087158203125
train RMSE: 5.25370454788208
train RMSE: 5.249332904815674
train RMSE: 5.2449727058410645
train RMSE: 5.24062442779541
train RMSE: 5.236286640167236
train RMSE: 5

train RMSE: 4.452486515045166
train RMSE: 4.450268268585205
train RMSE: 4.448055744171143
train RMSE: 4.445849418640137
train RMSE: 4.443648815155029
train RMSE: 4.4414544105529785
train RMSE: 4.439266204833984
train RMSE: 4.4370832443237305
train RMSE: 4.434906482696533
train RMSE: 4.432735443115234
train RMSE: 4.430570602416992
train RMSE: 4.42841100692749
train RMSE: 4.426257610321045
train RMSE: 4.424109935760498
train RMSE: 4.421968460083008
train RMSE: 4.419832229614258
train RMSE: 4.417701721191406
train RMSE: 4.415577411651611
train RMSE: 4.413458347320557
train RMSE: 4.4113450050354
train RMSE: 4.409237861633301
train RMSE: 4.407135963439941
train RMSE: 4.4050397872924805
train RMSE: 4.402949333190918
train RMSE: 4.400864601135254
train RMSE: 4.39878511428833
train RMSE: 4.396711349487305
train RMSE: 4.394643306732178
train RMSE: 4.392580986022949
train RMSE: 4.390524387359619
train RMSE: 4.388472557067871
train RMSE: 4.38642692565918
train RMSE: 4.3843865394592285
train RMSE:

train RMSE: 4.017343044281006
train RMSE: 4.016285419464111
train RMSE: 4.01522970199585
train RMSE: 4.014177322387695
train RMSE: 4.013127326965332
train RMSE: 4.01207971572876
train RMSE: 4.011035919189453
train RMSE: 4.009993553161621
train RMSE: 4.0089545249938965
train RMSE: 4.007917881011963
train RMSE: 4.0068840980529785
train RMSE: 4.005852699279785
train RMSE: 4.004824161529541
train RMSE: 4.003798007965088
train RMSE: 4.002774715423584
train RMSE: 4.001753807067871
train RMSE: 4.000735759735107
train RMSE: 3.999720573425293
train RMSE: 3.9987072944641113
train RMSE: 3.997696876525879
train RMSE: 3.9966890811920166
train RMSE: 3.9956839084625244
train RMSE: 3.9946811199188232
train RMSE: 3.993680953979492
train RMSE: 3.992683172225952
train RMSE: 3.9916882514953613
train RMSE: 3.9906957149505615
train RMSE: 3.989705801010132
train RMSE: 3.988718032836914
train RMSE: 3.9877328872680664
train RMSE: 3.986750364303589
train RMSE: 3.9857704639434814
train RMSE: 3.984792470932007
tr

train RMSE: 3.8048877716064453
train RMSE: 3.804311513900757
train RMSE: 3.8037354946136475
train RMSE: 3.8031604290008545
train RMSE: 3.802586317062378
train RMSE: 3.802013635635376
train RMSE: 3.801441192626953
train RMSE: 3.800869941711426
train RMSE: 3.800299644470215
train RMSE: 3.7997305393218994
train RMSE: 3.799161911010742
train RMSE: 3.7985942363739014
train RMSE: 3.798027515411377
train RMSE: 3.7974612712860107
train RMSE: 3.796896457672119
train RMSE: 3.7963321208953857
train RMSE: 3.795769214630127
train RMSE: 3.7952065467834473
train RMSE: 3.794645309448242
train RMSE: 3.794084310531616
train RMSE: 3.7935245037078857
train RMSE: 3.7929656505584717
train RMSE: 3.792407512664795
train RMSE: 3.7918500900268555
train RMSE: 3.7912933826446533
train RMSE: 3.7907376289367676
train RMSE: 3.790182590484619
train RMSE: 3.789628744125366
train RMSE: 3.7890753746032715
train RMSE: 3.788522481918335
train RMSE: 3.787971019744873
train RMSE: 3.7874200344085693
train RMSE: 3.78687000274

### Evulation

## AutoRec

### Data -preparation

In [14]:
from src.data_prep import movielens_prep
train, test = movielens_prep(1)
train

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### model

In [15]:
from src.autorec import AutoRec
num_items = train.shape[1]
model = AutoRec(num_hidden=500, num_features=num_items).to(device)

### train

In [17]:
from src.autorec import autorec_train
p = autorec_train(train, model,device, epochs=100)

epoch: 1 train RMSE: 0.7966745495796204
epoch: 2 train RMSE: 0.779500424861908
epoch: 3 train RMSE: 0.7672896385192871
epoch: 4 train RMSE: 0.7606155872344971
epoch: 5 train RMSE: 0.7539225816726685
epoch: 6 train RMSE: 0.7460877299308777
epoch: 7 train RMSE: 0.7486531734466553
epoch: 8 train RMSE: 0.7376992702484131
epoch: 9 train RMSE: 0.7308751940727234
epoch: 10 train RMSE: 0.7251607775688171
epoch: 11 train RMSE: 0.7202025055885315
epoch: 12 train RMSE: 0.7154077887535095
epoch: 13 train RMSE: 0.7114124894142151
epoch: 14 train RMSE: 0.7076403498649597
epoch: 15 train RMSE: 0.7043794989585876
epoch: 16 train RMSE: 0.7024579048156738
epoch: 17 train RMSE: 0.6983551979064941
epoch: 18 train RMSE: 0.7000725865364075
epoch: 19 train RMSE: 0.7031369805335999
epoch: 20 train RMSE: 0.6952686905860901
epoch: 21 train RMSE: 0.6939480900764465
epoch: 22 train RMSE: 0.690342903137207
epoch: 23 train RMSE: 0.6873338222503662
epoch: 24 train RMSE: 0.6857529282569885
epoch: 25 train RMSE: 0.684

### Evulation

## Comperison

In [30]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler((0, 5))
# scaler.fit_transform(torch.Tensor.cpu(p).detach().numpy())